# Use the `Inversion` class

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import inversion_ideas as ii
from regressor import LinearRegressor

## Synthetic data

In [ ]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

In [ ]:
# Build the X matrix
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [ ]:
# Generate synthetic data with noise
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
std_err =  1e-2 * maxabs
noise = rng.normal(scale=std_err, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

## Inversion with beta scheduling

In [ ]:
simulation = LinearRegressor(X, sleep=1)

In [ ]:
uncertainty = std_err * np.ones_like(synthetic_data)
data_misfit = ii.DataMisfit(synthetic_data, uncertainty, simulation)
data_misfit

In [ ]:
smallness = ii.TikhonovZero(n_params)
smallness

In [ ]:
# Define objective function with starting beta
beta_0 = 1e4
regularization = beta_0 * smallness
phi = data_misfit + regularization

# Initial model
initial_model = np.zeros(n_params)

# Minimizer
minimizer = ii.ConjugateGradient()

# Beta cooling
beta_cooler = ii.MultiplierCooler(cooling_factor=2.0)

# Stopping criteria
chi_target = 1.0
stopping_criteria = ii.ChiTarget(data_misfit, chi_target=chi_target)

# Inversion log
inversion_log = ii.create_standard_log(phi)

# Inversion
inversion = ii.Inversion(
    phi,
    initial_model,
    minimizer,
    stopping_criteria=stopping_criteria,
    log=inversion_log,
    cache_models=True,
)

In [ ]:
with inversion.log.show_live() as live:
    for model in inversion:
        # Cool down beta
        beta_cooler(regularization)

        # Refresh table
        live.refresh()

In [ ]:
inverted_model = inversion.model
inverted_model

In [ ]:
print("Result:")
print(inverted_model)
print()
print("True model:")
print(true_model)

In [ ]:
inversion.log.table

In [ ]:
df = pd.DataFrame(inversion.log.log).set_index("iter")
df

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(6, 8))
axes[0].plot(df.index, df.beta, "o-")
axes[0].set_ylabel("Beta")

axes[1].plot(df.index, df.phi_d, "o-")
axes[1].axhline(data_misfit.n_data, linestyle="--", color="grey")
axes[1].set_ylabel("Data misfit")
axes[1].set_yscale("log")

axes[2].plot(df.index, df.phi_m, "o-")
axes[2].set_ylabel("Model norm")

plt.show()

In [ ]:
inversion.models

### Manually running iterations

In [ ]:
# Define objective function with starting beta
beta_0 = 1e4
regularization = beta_0 * smallness
phi = data_misfit + regularization

# Inversion log
inversion_log = ii.create_standard_log(phi)

# Inversion
inversion = ii.Inversion(
    phi,
    initial_model,
    minimizer,
    stopping_criteria=stopping_criteria,
    log=inversion_log,
    cache_models=True,
)

In [ ]:
print(inversion.counter)

In [ ]:
model = next(inversion)
model

In [ ]:
print(inversion.counter)

### Break iterations at any point and continue

In [ ]:
# Define objective function with starting beta
beta_0 = 1e4
regularization = beta_0 * smallness
phi = data_misfit + regularization

# Inversion log
inversion_log = ii.create_standard_log(phi)

# Inversion
inversion = ii.Inversion(
    phi,
    initial_model,
    minimizer,
    stopping_criteria=stopping_criteria,
    log=inversion_log,
    cache_models=True,
)

In [ ]:
with inversion.log.show_live() as live:
    for model in inversion:
        # Cool down beta
        beta_cooler(regularization)

        # Refresh table
        live.refresh()

        if inversion.counter == 4:
            break

We can checkout the model, convergence curves, etc:

In [ ]:
print("Result:")
print(inversion.model)
print()
print("True model:")
print(true_model)

In [ ]:
df = pd.DataFrame(inversion.log.log).set_index("iter")

fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(6, 8))
axes[0].plot(df.index, df.beta, "o-")
axes[0].set_ylabel("Beta")

axes[1].plot(df.index, df.phi_d, "o-")
axes[1].axhline(data_misfit.n_data, linestyle="--", color="grey")
axes[1].set_ylabel("Data misfit")
axes[1].set_yscale("log")

axes[2].plot(df.index, df.phi_m, "o-")
axes[2].set_ylabel("Model norm")

plt.show()

We can then continue iterating manually

In [ ]:
model = next(inversion)
beta_cooler(regularization)
model

In [ ]:
inversion.log.table

In [ ]:
df = pd.DataFrame(inversion.log.log).set_index("iter")

fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(6, 8))
axes[0].plot(df.index, df.beta, "o-")
axes[0].set_ylabel("Beta")

axes[1].plot(df.index, df.phi_d, "o-")
axes[1].axhline(data_misfit.n_data, linestyle="--", color="grey")
axes[1].set_ylabel("Data misfit")
axes[1].set_yscale("log")

axes[2].plot(df.index, df.phi_m, "o-")
axes[2].set_ylabel("Model norm")

plt.show()

And we can keep iterating until stopping criteria is met:

In [ ]:
with inversion.log.show_live() as live:
    for model in inversion:
        # Cool down beta
        beta_cooler(regularization)

        # Refresh table
        live.refresh()

In [ ]:
print("Result:")
print(inversion.model)
print()
print("True model:")
print(true_model)

In [ ]:
df = pd.DataFrame(inversion.log.log).set_index("iter")

fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(6, 8))
axes[0].plot(df.index, df.beta, "o-")
axes[0].set_ylabel("Beta")

axes[1].plot(df.index, df.phi_d, "o-")
axes[1].axhline(data_misfit.n_data, linestyle="--", color="grey")
axes[1].set_ylabel("Data misfit")
axes[1].set_yscale("log")

axes[2].plot(df.index, df.phi_m, "o-")
axes[2].set_ylabel("Model norm")

plt.show()